In [2]:
%cd ../binfo1-work

/rna/seojin/proj/bioinformatics/bioinfo_term_proj/binfo1-work


In [2]:
!samtools mpileup ../binfo1-datapack1/CLIP-35L33G.bam > Seojin_Termproj/CLIP-35L33G.pileup
!wc -l CLIP-let7g.pileup

[mpileup] 1 samples in 1 input files
68548 CLIP-let7g.pileup


In [3]:
!head Seojin_Termproj/CLIP-35L33G.pileup

chr1	3056473	N	4	^!G^!G^!G^!G	HGGI
chr1	3056474	N	4	AAAA	HB@I
chr1	3056475	N	4	AAAA	GG?I
chr1	3056476	N	4	TTTT	HGGH
chr1	3056477	N	4	GGGG	HEGI
chr1	3056478	N	4	GGGG	GEEI
chr1	3056479	N	4	AAAA	EA=I
chr1	3056480	N	4	AAAA	@AEH
chr1	3056481	N	4	GGGG	GFFI
chr1	3056482	N	4	TTTT	G=BG


In [3]:
import pandas as pd

In [3]:
pileup = pd.read_csv('Seojin_Termproj/CLIP-35L33G.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])
pileup.tail()

/tmp/ipykernel_2155664/4078999649.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  pileup = pd.read_csv('Seojin_Termproj/CLIP-35L33G.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])


,chrom,pos,_ref,count,basereads,quals
959697295,MU069435.1,31107,N,2,GG,BC
959697296,MU069435.1,31108,N,2,GG,B>
959697297,MU069435.1,31109,N,2,TT,@3
959697298,MU069435.1,31110,N,2,GG,B1
959697299,MU069435.1,31111,N,2,G$G$,@6


Leave only match(. or ,), mismatch(ACGT or acgt), deletion(* or */#) from basereads. I won't consider insertion(+)

In [5]:
import re


In [6]:
# (+or-)(any number k)(length k with word with at leaste 1 element in ATGCatgc) to regex
pattern = re.compile(r'[\+\-]([1-9][0-9]*)((?=.*[ATGCatgc])[A-Za-z]{\1})')

for row in pileup.itertuples():
    # check if pattern is contained in the basereads
    match = pattern.search(row.basereads)
    if match:
        print(match.groups())

There is no -3ATG case. All of the deletion is marked as -3NNN format. 

In [7]:
toremove = re.compile('[^ATGCatgc\*]') 
pileup['filtered_basereads'] = pileup['basereads'].apply(lambda x: toremove.sub('', x))

<>:1: SyntaxWarning: invalid escape sequence '\*'
<>:1: SyntaxWarning: invalid escape sequence '\*'
/tmp/ipykernel_2155664/3979841347.py:1: SyntaxWarning: invalid escape sequence '\*'
  toremove = re.compile('[^ATGCatgc\*]')


In [8]:
import numpy as np
def Shannon_entropy(match):
    num_A = match.upper().count('A')
    num_T = match.upper().count('T')
    num_G = match.upper().count('G')
    num_C = match.upper().count('C')
    num_del = match.upper().count('*')

    # Calculate total number of valid bases
    total_bases = num_A + num_T + num_G + num_C + num_del

    # Calculate probabilities
    if total_bases > 0:
        p_A = num_A / total_bases
        p_T = num_T / total_bases
        p_G = num_G / total_bases
        p_C = num_C / total_bases
        p_del = num_del / total_bases

        # Calculate Shannon entropy
        entropy = 0
        for p in [p_A, p_T, p_G, p_C, p_del]:
            if p > 0:
                entropy -= p * np.log2(p)
    else:
        entropy = 0

    return entropy


In [13]:
def Calculate_Shannon_Entropy(pileup):
    for row in pileup.itertuples():
        shannon_entropy = Shannon_entropy(row.filtered_basereads)
        pileup.at[row.Index, 'Shannon_entropy'] = shannon_entropy

In [14]:
Calculate_Shannon_Entropy(pileup)

In [15]:
pileup.head()

,chrom,pos,_ref,count,basereads,quals,filtered_basereads,Shannon_entropy
0,chr1,3056473,N,4,^!G^!G^!G^!G,HGGI,GGGG,0.0
1,chr1,3056474,N,4,AAAA,HB@I,AAAA,0.0
2,chr1,3056475,N,4,AAAA,GG?I,AAAA,0.0
3,chr1,3056476,N,4,TTTT,HGGH,TTTT,0.0
4,chr1,3056477,N,4,GGGG,HEGI,GGGG,0.0


In [17]:
pileup.to_csv('Seojin_Termproj/CLIP-35L33G_with_Shannon.pileup', sep='\t', index=False)

In [ ]:
%wget https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M35/GRCm39.primary_assembly.genome.fa.gz
%gunzip GRCm39.primary_assembly.genome.fa.gz
%samtools faidx GRCm39.primary_assembly.genome.fa

In [20]:
mutation_rich = pileup[(pileup['Shannon_entropy'] > 0.8) & (pileup['count'] > 50)]


In [24]:
len(mutation_rich)
mutation_rich.head()
mutation_rich.to_csv('Seojin_Termproj/CLIP-35L33G_with_Shannon_mutation_rich.pileup', sep='\t', index=False)

In [5]:
mutation_rich = pd.read_csv('Seojin_Termproj/CLIP-35L33G_with_Shannon_mutation_rich.pileup', sep='\t')

In [6]:
with open('Seojin_Termproj/CLIP-35L33G_mutation_rich.bed', 'w') as f:
    for row in mutation_rich.itertuples():
        f.write(f'{row.chrom}\t{row.pos-6}\t{row.pos+5}\n')

In [ ]:
%bedtools getfasta -fi GRCm39.primary_assembly.genome.fa -bed CLIP-35L33G_mutation_rich.bed -fo CLIP-35L33G_mutation_rich_sequences.fa
%sed '/^>/d' CLIP-35L33G_mutation_rich_sequences.fa |  grep -E '^.{5}G' > CLIP-35L33G_mutation_rich_sequences_without_name_with_G_6left_5right.txt 
%sed 's/T/U/g' CLIP-35L33G_mutation_rich_sequences_without_name_with_G_6left_5right.txt > CLIP-35L33G_mutation_rich_sequences_without_name_with_G_6left_5right_TtoU.txt 

https://weblogo.threeplusone.com

Used above weblogo site(WebLogo 3>Create your own logos) to create weblogo image for Figure2A in the article. I uploaded  
CLIP-35L33G_mutation_rich_sequences_without_name_with_G_6left_5right_TtoU.txt  
file to the sequence data input of the WebLogo site. 

Option 
- First position number: -5
- Color scheme: Classic (NA)
- Units: bits, probability
- Version fineprint: off
- Error bar: off


I manually merged the two figures bits and probabilty in the powerpoint slide. 

Sort hexamer by frequency

In [ ]:
%awk '{count[substr($0, 4, 6)]++} END {for (seq in count) {print seq, count[seq]}}' CLIP-35L33G_mutation_rich_sequences_without_name_with_G_6left_5right_TtoU.txt | sort -k2 -rn | awk '{a[FNR]=$0; sum+=$2} END {for (i=1; i<=FNR; i++) {split(a[i], b, " "); print b[1], b[2], (b[2]/sum)*100}}' > top_LIN28A_interacting_hexamer.txt

In [7]:
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN
import pandas as pd
def my_round(in_number, ndigits=0, rounding_rule=1):
    """rounding_rule==1: ROUND_HALF_UP, Round to nearest, ties away from zero
       rounding_rule==2: ROUND_HALF_EVEN, Round to nearest, ties to even """
    # make float expression
    if ndigits > 0:
        expression = '0.' + '0'*ndigits
        number = in_number
    else: # 0 or negative
        expression = '0'
        number = in_number / (10**(-ndigits))
    
    # round by rounding rule
    if rounding_rule == 2:
        round_number = Decimal(number).quantize(Decimal(expression), rounding=ROUND_HALF_EVEN)
    else:
        round_number = Decimal(number).quantize(Decimal(expression), rounding=ROUND_HALF_UP)
    
    # return number
    if ndigits > 0:
        return float(round_number)
    else: # 0 or negative
        return int(round_number * (10**(-ndigits)))

In [15]:
lin28a_interacting_hexamer = pd.read_csv("/rna/seojin/proj/bioinformatics/bioinfo_term_proj/binfo1-work/Seojin_Termproj/top_LIN28A_interacting_hexamer.txt", sep=" ", names=["Sequence", "Count", "Frequency (%)"])

In [16]:
for row in lin28a_interacting_hexamer.itertuples():
    lin28a_interacting_hexamer.at[row.Index, 'Frequency (%)'] = my_round(row._3, 2)

In [22]:
# print only Sequence and Frequency columns
lin28a_interacting_hexamer[['Sequence', 'Frequency (%)']].head(10)

# Save this to table without index
lin28a_interacting_hexamer[['Sequence', 'Frequency (%)']].head(10).to_csv('Seojin_Termproj/top10_LIN28A_interacting_hexamer.txt', sep=' ', index=False)

,Sequence,Frequency (%)
0,AAGGAG,2.29
1,AAGAAG,1.68
2,AAGAGA,1.39
3,GAGGAG,1.35
4,AAGAGG,1.17
5,AAGGUG,1.15
6,GAGAGA,0.89
7,AAGAAA,0.86
8,UGGAGA,0.84
9,GAGAAG,0.83


I made table on powerpoint using the above data

For sequence flanking hexamer binding to LIN28A, I used below code

In [ ]:
%awk '{count[substr($0, 2, 2)"......"substr($0, 10, 2)]++} END {for (seq in count) {print seq, count[seq]}}' CLIP-35L33G_mutation_rich_sequences_without_name_with_G_6left_5right_TtoU.txt | sort -k2 -rn | awk '{a[FNR]=$0; sum+=$2} END {for (i=1; i<=FNR; i++) {split(a[i], b, " "); print b[1], b[2], (b[2]/sum)*100}}' > top_LIN28A_flanking_hexamer.txt    

In [25]:
lin28a_flanking = pd.read_csv("/rna/seojin/proj/bioinformatics/bioinfo_term_proj/binfo1-work/Seojin_Termproj/top_LIN28A_flanking_hexamer.txt", sep=" ", names=["Sequence", "Count", "Frequency (%)"])

In [26]:
for row in lin28a_flanking.itertuples():
    lin28a_flanking.at[row.Index, 'Frequency (%)'] = my_round(row._3, 2)

In [28]:
lin28a_flanking[['Sequence', 'Frequency (%)']].head(10)
lin28a_flanking[['Sequence', 'Frequency (%)']].head(10).to_csv('Seojin_Termproj/top10_LIN28A_flanking.txt', sep=' ', index=False)

,Sequence,Frequency (%)
0,CA......UG,1.41
1,CC......GG,1.19
2,CU......AG,1.00
3,GG......CC,0.97
4,GA......UG,0.95
5,AA......AA,0.91
6,AG......CU,0.91
7,UU......AA,0.88
8,AA......UU,0.87
9,UC......GA,0.84


To normalize the kmer with the frequency of occurence in reference genome data. I used jellyfish to count the k-mer(in this case, hexamer) in the Reference.

In [ ]:
%jellyfish count -m 6 -s 100M -t 10 GRCm39.primary_assembly.genome.fa
%jellyfish dump mer_counts.jf > mer_counts_dumps.fa 

ModuleNotFoundError: No module named 'Bio'

In [ ]:
%mamba install biopython


In [32]:
from Bio import SeqIO

In [59]:
sum = 0
with open('Seojin_Termproj/mer_counts_dumps_frequency.fa', 'w') as write_file:
    with open('Seojin_Termproj/mer_counts_dumps.fa') as f:
        records = list(SeqIO.parse(f, "fasta"))
        for record in records:
            sum += int(record.id)
        write_file.write(f'Sequences\tCounts\tFrequency (%)\n')
        for record in records:
            write_file.write(f'{record.seq}\t{int(record.id)}\t{int(record.id)/sum*100}\n')


In [57]:
%sort -nr -k2,2 mer_counts_dumps_frequency.fa > mer_counts_dumps_frequency_sorted.fa 

UsageError: Line magic function `%sort` not found.


In [74]:
lin28a_hexamer = pd.read_csv("Seojin_Termproj/top_LIN28A_interacting_hexamer.txt", sep=" ", names=["Sequence", "Count", "Frequency (%)"])
jellyfish_count = pd.read_csv("Seojin_Termproj/mer_counts_dumps_frequency_sorted.fa", sep="\t", names=["Sequence", "Count", "Frequency (%)"])

In [75]:
for row in lin28a_hexamer.itertuples():
    genome_freq = float(jellyfish_count[jellyfish_count['Sequence'] == row.Sequence.replace('U', 'T')]['Frequency (%)'].values[0])
    lin28a_hexamer.at[row.Index, 'GRCm39 Genome Frequency (%)'] = genome_freq
    lin28a_hexamer.at[row.Index, 'Motif Enrichment (fold)'] = row._3 / genome_freq


In [76]:
# sort by Motif Enrichment (fold) in descending order
lin28a_hexamer.sort_values(by='Motif Enrichment (fold)', ascending=False, inplace=True)

In [77]:
lin28a_hexamer.head()

,Sequence,Count,Frequency (%),GRCm39 Genome Frequency (%),Motif Enrichment (fold)
505,UCGCGA,8,0.031934,0.000392,81.370296
108,AAGGCG,69,0.275427,0.004820,57.141051
458,ACGACG,10,0.039917,0.000757,52.750133
433,ACGGCG,11,0.043909,0.000843,52.096592
0,AAGGAG,573,2.287240,0.048228,47.425716
